# Text Analytics Coursework -- Tasks 1.1 to 1.3

# Instructions:

This notebook is forms the first part of your coursework assignment for Text Analytics in Spring 2025. You will need to read the instructions below and complete numbered tasks indicated by "TASK n". To complete the tasks, you will write code or explanations between the comments "#WRITE YOUR ANSWER HERE" and "#END OF ANSWER". For example:

TASK 0: Complete the function below to output "hello world". 


In [2]:
def demo_fun():
    # WRITE YOUR ANSWER HERE
    print("hello world")
    # END OF ANSWER

### DO NOT MODIFY
demo_fun()
###

hello world


There is also some code in the cell that should not be modified. This code saves your results to file in the correct format, which is necessary for us to be able to mark your answers. Before you submit your notebook, please make sure this code has not been modified, then restart your kernel, clear all cell outputs, run all of your code once again, then save the notebook. 

Please note:
  * The notebook you upload must include all the saved cell output after running all cells.
  * The notebook code must be complete so that it reproduces all your output when we run it. 
  * For this coursework, we recommend that you use your virtual environment that you created for the labs. The packages you need are: numpy, scipy, nltk, pytorch, transformers and datasets (from HuggingFace), pandas, matplotlib and scikit-learn. 

## Marking guidelines:
1. This notebook is worth 32% of the marks for the Text Analytics assignment.
1. The number of marks for each task is shown alongside the task.
1. We will evaluate the output of your code after running it, and marks will be awarded based on how well the output matches the task's instructions. 
1. We will give partial marks for incomplete or partially correct answers. 
1. We do not give additional marks for code style or comments, but clear code will help us to understand what you have done so that we can award partial marks where necessary. 
1. Unless the task asks you to implement something from scratch, there is no penalty for using software libraries in your implementation.

## Support:

The main source of support will be during the lab sessions. The TAs and lecturers will help you with questions about the lectures, the code provided for you in this notebook, and general questions about the topics we cover. For the assessed tasks, they can only answer clarifying questions about what you have to do. Please email Edwin if you have any other queries edwin.simpson@bristol.ac.uk and/or post your query to the Teams channel for this unit.

## Deadline:

The notebook must be submitted along with the second notebook on Blackboard before **Monday 28th April at 13.00**. 

## Submission:

For this part of the assignment, please zip up the folder containing this file and the 'outputs' directory, containing the output from this notebook as .csv files. Please name your notebook file like this:
   * Name this notebook 'text_analytics_part1_\<student number\>.ipynb'. Replace '\<student number\>' with your student number, which consists only of digits beginning with '2'. 
   * We mark anonymously, so please don't include your name in the notebook.

You can submit the file on Blackboard to the submission point "Text Analytics Part 1 Notebook". Remember that the assignment also has parts 2 and 3, described in the PDF file on Blackboard.

# Setup: random seeds

Each student will work with slightly different data splits and model weights, which will be determined by setting your 'random seed'. 
We will check that your results come from using your random seed. Please set the seed in the cell below by changing the value of 'my_student_number' to your own student number (not your username, the number you can see on eVision that contains only digits). 

Using the correct seed ensures that your results are reproducible when we rerun your notebook.

In [3]:
import torch
import random
import numpy as np

def set_seed(seed: int = 42):
    random.seed(seed)  # Python's built-in random module
    np.random.seed(seed)  # NumPy
    torch.manual_seed(seed)  # PyTorch CPU
    torch.cuda.manual_seed(seed)  # PyTorch GPU (if available)
    torch.cuda.manual_seed_all(seed)  # Multi-GPU
    torch.backends.cudnn.deterministic = True  # Ensure deterministic behavior
    torch.backends.cudnn.benchmark = False  # Disable benchmark mode for reproducibility

### SET YOUR SEED TO YOUR STUDENT NUMBER HERE
my_student_number = 2604090
set_seed(my_student_number)

# Setup: loading the data

Let's make a folder to save the output of your work:

In [4]:
import os
import pandas as pd
os.makedirs('./outputs', exist_ok=True)


Now, let's load some more packages we will need later:

In [5]:
%load_ext autoreload
%autoreload 2

# Use HuggingFace's datasets library to access the Emotion dataset
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split 


The dataset classifies paragraphs taken from corporate disclosures that discuss climate-related issues. It classifiers them into "risk" (0), "neutral" (1) or "opportunity" (2) representing the sentiment of the paragraph.

First we need to load the data. The data is already split into train, validation and test. The _validation_ set (also called 'development' set or 'devset') can be used to compute performance of your model when tuning hyperparameters, optimising combinations of features, or looking at the errors your model makes before improving it. This allows you to hold out the test set (i.e., not to look at it at all when developing your method) to give a fair evaluation of the model and how well it generalises to new examples. This avoids tuning the model to specific examples in the test set. An alternative approach to validation is to not use a single fixed validation set, but instead use [cross validation](https://scikit-learn.org/stable/modules/cross_validation.html). 

In [6]:
cache_dir = "./data_cache"

# load the original training set from HuggingFace
train_dataset = load_dataset(
    "climatebert/climate_sentiment",
    split="train",
    cache_dir=cache_dir,
)

# we're going to create a new validation set by splitting the data
dataset_splits = train_dataset.train_test_split(test_size=0.2)
train_dataset = dataset_splits["train"]
val_dataset = dataset_splits["test"]

train_texts = np.array(train_dataset["text"])
val_texts = np.array(val_dataset["text"])

train_labels = np.array(train_dataset["label"])
val_labels = np.array(val_dataset["label"])

print(f"Training dataset with {len(train_texts)} instances loaded")
print(f"Development/validation dataset with {len(val_texts)} instances loaded")

### DO NOT MODIFY
# save gold labels to file
pd.DataFrame(val_labels).to_csv('./outputs/val_labels.csv')

Training dataset with 800 instances loaded
Development/validation dataset with 200 instances loaded


In this notebook, you're going to build three different classifiers for this dataset, then compare how they work, and analyse the results. We are going to start by implementing a naïve Bayes classifier from scratch. 

We are going to begin by initialising some useful variables and doing some very simple pre-processing using CountVectorizer.

In [7]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import word_tokenize

num_classes = 3

def preprocess(train_texts):
    vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=word_tokenize)
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess(train_texts)

C:\Users\suhas\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## TASK 1.1a

Complete the function below to compute the class priors, $p(y_n = c)$ for each class label $c$, where $y_n$ is the class label of document $n$. Do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the class priors as a list or Numpy array containing the probabilities. You do not need to apply any smoothing or regularisation.    (3 marks)

In [8]:
def compute_class_priors(texts, labels):
    priors = np.zeros(num_classes)
### WRITE YOUR ANSWER HERE
    for c in range(num_classes):
        priors[c] = np.sum(labels == c)
    priors = priors / len(labels)
    ### END OF ANSWER
    return priors


class_priors = compute_class_priors(train_texts, train_labels)
print(class_priors)

### DO NOT MODIFY
pd.DataFrame(class_priors).to_csv('./outputs/11a_class_priors.csv')

[0.36    0.39375 0.24625]


## TASK 1.1b

Complete the function below to extract n-gram features from the text, then compute the liklihood $p(x_{ni} = w | y_n = c)$ that the $i$ th n-gram in document $n$ is $w$, given that the class of $n$ is $c$. Again, do not use the Sklearn implementation to do this, but implement it yourself, e.g., using Numpy functions. The function must output the likelihoods as a 2D Numpy array containing probabilities. You should apply smoothing by adding counts of +1 to the counts of each feature.  (3 marks)

In [9]:
def compute_feature_likelihoods(X, labels):

    likelihoods = np.ones((num_features, num_classes))  # a 2D numpy array where you can store the likelihoods. Note that all values are initialised to one.

    ### WRITE YOUR ANSWER HERE
    for c in range(num_classes):
        X_c = X[labels == c]
        feature_count = np.sum(X_c, axis=0)
        total_count = np.sum(feature_count) + num_features
        likelihoods[:, c] = (feature_count + 1) / total_count


    ### END OF ANSWER
    return likelihoods

likelihoods = compute_feature_likelihoods(X, train_labels)
print("Likelihoods shape:", likelihoods.shape)


### DO NOT MODIFY
pd.DataFrame(likelihoods).to_csv('./outputs/11b_likelihoods.csv')

Likelihoods shape: (34111, 3)


Now, we are going to use the code in the next cell to compute the log probabilities of each class for each text in the validation set. This code will use the previous functions you implemented, compute_class_priors and compute_feature_likelihoods. The log probabilities will be stored in the 'predictions' array.

In [10]:
from scipy.special import logsumexp


def NB_classify(class_priors, likelihoods, X_val):

    predictions = np.zeros((X_val.shape[0], num_classes))  # an empty numpy array to store the predictions in

    sum_of_log_likelihoods = X_val.dot(np.log(likelihoods))
    log_joint_prob = sum_of_log_likelihoods + np.log(class_priors)[None, :]
    for n, doc in enumerate(X_val):
        predictions[n, :] = log_joint_prob[n]
        predictions[n, :] -= logsumexp(predictions[n, :])
    return predictions

predictions = NB_classify(class_priors, likelihoods, X_val)
print(predictions)

[[-2.12779066e-05 -1.07578732e+01 -2.15170768e+01]
 [-4.07290500e-04 -7.80618754e+00 -2.50005580e+01]
 [-9.72928694e-06 -1.15403748e+01 -4.21595116e+01]
 [-1.58537900e+01 -1.31178581e-07 -2.08002012e+01]
 [-1.15778182e-05 -1.20113457e+01 -1.21102434e+01]
 [-3.10613904e+01 -5.68434189e-14 -3.23864882e+01]
 [-1.46615434e-07 -2.12794292e+01 -1.57393715e+01]
 [-1.67035717e+01 -2.49234272e+01 -5.56990472e-08]
 [-1.02819664e+01 -7.71844701e-01 -6.20257342e-01]
 [-5.06280125e+01 -4.16234288e+01  0.00000000e+00]
 [-2.84751075e+01 -3.66640052e-11 -2.40404033e+01]
 [-8.95842186e+00 -1.76858468e-04 -9.94028204e+00]
 [-2.54735501e+01 -1.29822142e+01 -2.30090205e-06]
 [-1.01107379e+01 -4.09721308e-04 -7.90472319e+00]
 [ 0.00000000e+00 -3.19761479e+01 -3.43999238e+01]
 [-1.81925567e-02 -4.04736712e+00 -7.48799346e+00]
 [-1.17833761e+01 -8.42690076e-06 -1.40430262e+01]
 [-1.80043894e-01 -1.89287196e+00 -4.25960431e+00]
 [-4.43063370e-03 -5.42275051e+00 -1.20496788e+01]
 [-6.23033865e+00 -1.97174524e-

Use the 'predictions' array above to compute and print the accuracy of the classifier on the validation set.   

In [11]:
from sklearn.metrics import accuracy_score

accuracy_score(val_labels, np.argmax(predictions, axis=1))

0.73

## TASK 1.1c

The simplicty of naïve Bayes means that we can quite easily interpret the model. In the code above, we used the functions you implemented, compute_feature_likelihoods and compute_class_priors, to train an NB classifier with our training set. Given this classifier, which are the five n-gram features that most strongly indicate that the document belongs to class 0? Store these features in the 'top_features' list below.    (4 marks)

In [12]:
top_features = []

### WRITE YOUR ANSWER HERE

log_likelihoods = np.log(likelihoods)
scores = log_likelihoods[:, 0] - np.max(log_likelihoods[:, 1:], axis=1)
top_indices = np.argsort(scores)[-5:][::-1]
feature_names = np.array(vectorizer.get_feature_names_out())
top_features = feature_names[top_indices].tolist()

### END OF ANSWER

### DO NOT MODIFY
print(top_features)
pd.DataFrame(top_features).to_csv('./outputs/11c_top_feats.csv')
###

['physical risks', 'weather events', 'extreme weather', 'caused by', 'impact our']


Up to this point, the classifier used bigrams features extracted using CountVectorizer with the wordnet tokenizer. 

## TASK 1.1d

Your task is to improve the naïve Bayes classifier by changing the preprocessing or features only. It is up to you to decide how many changes are needed to improve the classifier -- a single change may be enough to achieve a good result (and maximum marks) and you should only include steps that help performance. Complete the 'preprocess_improved' function below, and run the cell to compute accuracy of the improved classifier on the validation set.     (3 marks)

In [13]:
def preprocess_improved(train_texts):
    ### WRITE YOUR ANSWER HERE

    vectorizer = CountVectorizer(ngram_range=(1,2), tokenizer=word_tokenize,stop_words='english')
    X = vectorizer.fit_transform(train_texts).toarray()
    num_features = X.shape[1]

    X_val = vectorizer.transform(val_texts).toarray()

    ### END OF ANSWER

    return X, X_val, vectorizer, num_features

X, X_val, vectorizer, num_features = preprocess_improved(train_texts)
class_priors = compute_class_priors(train_texts, train_labels)
likelihoods = compute_feature_likelihoods(X, train_labels)
predictions = NB_classify(class_priors, likelihoods, X_val)
predictions_nb = np.argmax(predictions, axis=1)

### DO NOT MODIFY
pd.DataFrame(predictions_nb).to_csv('./outputs/11d_improved_preds.csv')
accuracy_improved = accuracy_score(val_labels, predictions_nb)
print(accuracy_improved)
###

C:\Users\suhas\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


0.77


## TASK 1.2

Below is an implementation of a neural network classifier that we can apply to the same dataset. However, there are some mistakes in the code and some poor choices in the choice of parameters and architecture. Your task is to fix the errors, make better parameter choices, and improve the model's performance. **Modify the code within the next cell** to improve the neural network classifier, then run it and compute its accuracy using the code in the cell after that.   (8 marks)

In [16]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE: MODIFY THE CODE WITHIN THIS CELL

from torch import nn
from torch import optim
from transformers import AutoTokenizer
from torch.utils.data import DataLoader

tokenizer = AutoTokenizer.from_pretrained("climatebert/distilroberta-base-climate-f")  

sequence_length = 64  # truncate all docs longer than this. Pad all docs shorter than this.
batch_size = 32

def tokenize_function(examples):
    return tokenizer(
        examples["text"],  # Adjust the key based on your dataset structure
        padding="max_length",  # Ensures equal sequence lengths
        truncation=True,       # Truncates longer sequences
        max_length=sequence_length,        # Adjust as needed
        return_tensors="pt"
    )

tokenized_dataset = train_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])  # Adjust column names
train_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=True)

tokenized_dataset = val_dataset.map(tokenize_function, batched=True)
tokenized_dataset.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])  # Adjust column names
val_loader = DataLoader(tokenized_dataset, batch_size=batch_size, shuffle=False)

class FFTextClassifier(nn.Module):

    def __init__(self, vocab_size, sequence_length, num_classes, embedding_size=256, hidden_size=128):
        super(FFTextClassifier, self).__init__()

        self.embedding_size = embedding_size
        self.sequence_length = sequence_length

        self.embedding_layer = nn.Embedding(vocab_size, embedding_size)  # embedding layer
        self.dropout = nn.Dropout(0.5)  # added dropout for regularization
        self.hidden_layer = nn.Linear(embedding_size, hidden_size)  # note: we now use pooled input, not flattened
        self.activation = nn.ReLU()
        self.output_layer = nn.Linear(hidden_size, num_classes)

    def forward(self, input_words):
        embedded_words = self.embedding_layer(input_words)  # (batch, seq_len, embedding_size)

        # Create padding mask from input ids (assumes padding token = 0)
        mask = (input_words != 0).unsqueeze(-1).float()  # (batch, seq_len, 1)
        masked_embed = embedded_words * mask
        summed = masked_embed.sum(dim=1)  # sum over tokens
        counts = mask.sum(dim=1).clamp(min=1)
        pooled = summed / counts  # mean of non-padding embeddings

        z = self.hidden_layer(pooled)
        h = self.activation(z)
        h = self.dropout(h)
        output = self.output_layer(h)

        return output

    def run_training(self, train_dataloader, dev_dataloader):

        num_epochs = 10
        learning_rate = 1e-3

        loss_fn = nn.CrossEntropyLoss()
        optimizer = optim.Adam(self.parameters(), lr=learning_rate, weight_decay=1e-5)
        dev_losses = []

        for e in range(num_epochs):
            train_losses = []
            self.train()

            for i, batch in enumerate(train_dataloader):
                optimizer.zero_grad()
                output = self(batch['input_ids'])
                batch_loss = loss_fn(output, batch['label'])
                batch_loss.backward()
                optimizer.step()
                train_losses.append(batch_loss.item())

            print("Epoch: {}/{}".format((e+1), num_epochs),
                  "Training Loss: {:.4f}".format(np.mean(train_losses)))

            self.eval()
            val_losses_epoch = []

            for dev_batch in dev_dataloader:
                dev_output = self(dev_batch['input_ids'])
                dev_loss = loss_fn(dev_output, dev_batch['label'])
                val_losses_epoch.append(dev_loss.item())

            dev_losses.append(np.mean(val_losses_epoch))

            print("Epoch: {}/{}".format((e+1), num_epochs),
                  "Validation Loss: {:.4f}".format(dev_losses[-1]))

        return dev_losses

def predict_nn(trained_model, data_loader):
    trained_model.eval()
    pred_labs = []

    for batch in data_loader:
        test_output = trained_model(batch['input_ids'])
        predicted_labels = test_output.argmax(1)
        pred_labs.extend(predicted_labels.tolist())

    return pred_labs

vocab_size = max(tokenizer.get_vocab().values()) + 1
nn_classifier_model = FFTextClassifier(vocab_size, sequence_length, num_classes)
dev_losses = nn_classifier_model.run_training(train_loader, val_loader)

predictions_nn = predict_nn(nn_classifier_model, val_loader)

### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_nn).to_csv("./outputs/12_nn_preds.csv")
accuracy_nn = accuracy_score(val_labels, predictions_nn)
print(accuracy_nn)
###


Epoch: 1/10 Training Loss: 1.0667
Epoch: 1/10 Validation Loss: 1.0517
Epoch: 2/10 Training Loss: 0.9943
Epoch: 2/10 Validation Loss: 1.0168
Epoch: 3/10 Training Loss: 0.9146
Epoch: 3/10 Validation Loss: 0.9529
Epoch: 4/10 Training Loss: 0.8105
Epoch: 4/10 Validation Loss: 0.8875
Epoch: 5/10 Training Loss: 0.7047
Epoch: 5/10 Validation Loss: 0.8386
Epoch: 6/10 Training Loss: 0.5776
Epoch: 6/10 Validation Loss: 0.7836
Epoch: 7/10 Training Loss: 0.4906
Epoch: 7/10 Validation Loss: 0.7586
Epoch: 8/10 Training Loss: 0.4076
Epoch: 8/10 Validation Loss: 0.7508
Epoch: 9/10 Training Loss: 0.3459
Epoch: 9/10 Validation Loss: 0.7340
Epoch: 10/10 Training Loss: 0.2681
Epoch: 10/10 Validation Loss: 0.7537
0.685


We now explore the use of transformers for building a text classifier. First, let's look at how the process a document. We'll chose one at random from the training set:

In [17]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]

## TASK 1.3a

Use the HuggingFace transformers library to load the pretrained BERT model "prajjwal1/bert-tiny". Obtain a document embedding for the chosen document given above. Comment your code to explain how it obtains a representation of the document.    (3 marks)

In [19]:
### DO NOT MODIFY
chosen_document = train_texts[np.random.randint(len(train_texts))]
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModel
import torch

# Loading the tokenizer for the pretrained BERT model
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")

# Loading the BERT model for generating hidden representations
model = AutoModel.from_pretrained("prajjwal1/bert-tiny")

# Tokenizing the chosen document and converting it into input tensors
inputs = tokenizer(chosen_document, return_tensors="pt", truncation=True, padding='max_length', max_length=512)

# Passing the inputs through the model to get the output embeddings
with torch.no_grad():
    outputs = model(**inputs)

# Taking the embedding of the [CLS] token as the overall document representation
cls_embedding = outputs.last_hidden_state[:,0,:]

# Storing the document embedding in the required variable
doc_emb = []  # useing this variable to store the document embedding
doc_emb = cls_embedding.squeeze().tolist()
### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(doc_emb).to_csv('./outputs/13a_sen_emb.csv')
print(doc_emb)
###


[0.4747985303401947, 1.2661957740783691, -3.4104769229888916, -1.080893635749817, 0.39652612805366516, 0.4834730923175812, 0.30706390738487244, 1.4382047653198242, -0.19037285447120667, 0.5196099877357483, -0.25119709968566895, 0.7267217636108398, 0.8338472843170166, 0.0947263315320015, 0.6013916730880737, -1.729634165763855, -0.656603991985321, -0.8343584537506104, -1.1604818105697632, 1.4354984760284424, 0.6885497570037842, 0.9558952450752258, 2.655642032623291, 0.4720432460308075, 3.004547357559204, -0.8522915840148926, -0.48422691226005554, -0.3300834596157074, -0.16244348883628845, 1.4378259181976318, 0.0695171058177948, -0.8538330793380737, -0.648266077041626, -0.5296088457107544, -0.35218361020088196, -1.124722957611084, 0.20461539924144745, 0.40405794978141785, -1.4916101694107056, -0.17619588971138, 0.12367746233940125, 0.1453671008348465, 0.27699795365333557, -1.8794676065444946, -0.1967117190361023, -2.053748369216919, -1.270451545715332, 0.28574204444885254, 0.1134644895792

## TASK 1.3b

Using the same document embeddings method as the previous task, find the most similar document to the 'chosen_document' from within the validation set (from the 'val_texts' object). Use a standard similarity metric that considers the direction but not the magnitude of the embedding vectors. Use the same model as in task 1.8.  (2 marks)

In [22]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from sklearn.metrics.pairwise import cosine_similarity

# Getting the embedding for the chosen document using [CLS] token
inputs = tokenizer(chosen_document, return_tensors="pt", truncation=True, padding='max_length', max_length=512)
with torch.no_grad():
    outputs = model(**inputs)
chosen_emb = outputs.last_hidden_state[:,0,:].squeeze().unsqueeze(0)

# Creating a list to store embeddings for validation documents
val_embeddings = []

# Looping through each validation document and computing its embedding
for doc in val_texts:
    val_inputs = tokenizer(doc, return_tensors="pt", truncation=True, padding='max_length', max_length=512)
    with torch.no_grad():
        val_outputs = model(**val_inputs)
    val_cls_emb = val_outputs.last_hidden_state[:,0,:].squeeze()
    val_embeddings.append(val_cls_emb.numpy())

# Computing cosine similarities between the chosen document and validation documents
similarities = cosine_similarity(chosen_emb.numpy(), val_embeddings)

# Finding the index of the most similar document
most_similar_idx = similarities.argmax()

# Storing the most similar document
most_similar_doc = ""  # use this variable to store the most similar document
most_similar_doc = val_texts[most_similar_idx]
### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame([chosen_document, most_similar_doc]).to_csv("./outputs/13b_most_similar.csv")
print(chosen_document)
print(most_similar_doc)
###


We are developing an operational carbon accounting data infrastructure: a prototype that calculates carbon intensities and footprints across a variety of metrics (see metrics section for more details). The calculations are based on emissions data of scope 1 and 2, and selected scope 3 data.
Currently, all three private equity strategies diligence our current and prospective General Partners for Energy Efficiency, Water Usage and Natural Resource Usage as part of the Capital Dynamics R-EyeTM Rating. Our plan is to expand these factors over the next twelve months and formally consider measuring the results, once we have enough data for a meaningful analysis. The next phase of the plan will contemplate how to apply a weighting system to data collected from relevant Climate R-EyeTM Rating questions.


## TASK 1.3c

Implement a classifier based on the same pretrained transformer model, "prajjwal1/bert-tiny". Evaluate your model's performance on the validation set. Use an 'auto class' from HuggingFace to build your classifier (see https://huggingface.co/docs/transformers/model_doc/auto).   (6 marks)

In [35]:
### DO NOT MODIFY
set_seed(my_student_number)
###

### WRITE YOUR ANSWER HERE
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import AdamW
import torch
import numpy as np

# Tokenizing training and validation sets
tokenizer = AutoTokenizer.from_pretrained("prajjwal1/bert-tiny")
model = AutoModelForSequenceClassification.from_pretrained("prajjwal1/bert-tiny", num_labels=len(set(train_dataset["label"])))

train_enc = tokenizer(train_dataset["text"], padding=True, truncation=True, max_length=512, return_tensors="pt")
train_labels = torch.tensor(train_dataset["label"])
train_data = TensorDataset(train_enc["input_ids"], train_enc["attention_mask"], train_labels)
train_loader = DataLoader(train_data, batch_size=16, shuffle=True)

val_enc = tokenizer(val_dataset["text"], padding=True, truncation=True, max_length=512, return_tensors="pt")
val_labels = torch.tensor(val_dataset["label"])
val_data = TensorDataset(val_enc["input_ids"], val_enc["attention_mask"], val_labels)
val_loader = DataLoader(val_data, batch_size=32)

# Optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Training loop
model.train()
for epoch in range(3):
    for input_ids, attention_mask, labels in train_loader:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

# Evaluation
model.eval()
predictions_bert = []  # use this variable to store the predicted labels for the validation set
with torch.no_grad():
    for input_ids, attention_mask, labels in val_loader:
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        preds = torch.argmax(outputs.logits, dim=1)
        predictions_bert.extend(preds.tolist())
### END OF ANSWER

### DO NOT MODIFY
pd.DataFrame(predictions_bert).to_csv('./outputs/13c_bert_preds.csv')
accuracy_tinybert = accuracy_score(val_dataset["label"], predictions_bert)
print(accuracy_tinybert)
###


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at prajjwal1/bert-tiny and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


0.755
